In [16]:
import re
import nltk
import os
from langdetect import detect
from nltk.tokenize import word_tokenize
import pickle
import os
import pandas as pd
import io
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import coo_matrix

In [2]:
stop_words_file_nl = open('../data/misc_files/stopwords_dutch.txt', 'r')
stop_words_file_en = open('../data/misc_files/stopwords_english.txt', 'r')
stop_words_nl = stop_words_file_nl.read().split('\n')
stop_words_en = stop_words_file_en.read().split('\n')

In [3]:
texts_nl = []
texts_en = []
file_names_nl = []
file_names_en = []

In [11]:
source_path = '../data/text_files/clean/'

In [13]:
for file in os.listdir(source_path):
    with open(source_path + file, "r", encoding="utf-8") as infile:
        text = infile.readline()
        if detect(text) == 'en':
            texts_en.append(text)
        elif detect(text) == 'nl':
            texts_nl.append(text)

In [14]:
vectorizer_nl = TfidfVectorizer(ngram_range=(1, 3), stop_words=stop_words_nl)
vectorizer_en = TfidfVectorizer(ngram_range=(1, 3), stop_words=stop_words_en)
vec_nl = coo_matrix(vectorizer_nl.fit_transform(texts_nl))
vec_en = coo_matrix(vectorizer_en.fit_transform(texts_en))

C:\Users\s158079\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['opdat'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [31]:
df_nl = pd.DataFrame(vec_nl.toarray())

In [32]:
df_nl.columns = vectorizer_nl.get_feature_names()

In [33]:
df_nl['Doc_ID'] = df_nl.index

In [34]:
df_nl.head(1)

,aa,aa aa,aa aa aa,aa aa ar,aa aa contentid,aa aa ketenaanpak,aa aa ontucht,aa aa rn,aa aangeven,aa aangeven gemeente,...,zzp verdachte ontvangt,zzp vormen,zzp vormen bieden,zzp vragen,zzp vragen belastingen,zzp werken,zzp werken prostitutie,zzp zelfstandige,zzp zelfstandige personeel,Doc_ID
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [38]:
df_nl_flipped = df_nl.melt(id_vars = ["Doc_ID"], var_name="N-gram", 
        value_name="TF-IDF")

In [39]:
df_nl_flipped = df_nl_flipped.loc[df_nl_flipped['TF-IDF'] > 0]

In [43]:
df_nl_flipped.sort_values(by=['TF-IDF'], ascending=False)

,Doc_ID,N-gram,TF-IDF
230128616,36,slachtoffersm,0.651011
230128612,32,slachtoffersm,0.651011
230128650,70,slachtoffersm,0.598524
230128636,56,slachtoffersm,0.499071
128889900,136,kinderpornografie,0.474664
172070361,145,nvt,0.444772
253374281,109,true,0.380474
123818893,139,jv indd,0.372165
123818901,147,jv indd,0.372165
71828489,109,false,0.367539


In [44]:
fasa = df_nl_flipped.groupby(['N-gram'])['TF-IDF'].sum()

In [47]:
fasa.sort_values(ascending=False).head(50)

N-gram
mensenhandel                 15.060718
slachtoffers                  9.848426
seksueel                      5.864990
geweld                        5.535520
kinderen                      5.402990
jaar                          5.000537
uitbuiting                    4.849047
seksueel geweld               4.775982
rapporteur                    4.573483
nationaal                     4.530898
slachtoffer                   4.440166
nationaal rapporteur          4.151208
aantal                        4.088552
zaken                         3.965975
nederland                     3.847452
geweld kinderen               3.402717
aanpak                        3.400036
seksueel geweld kinderen      3.388596
onderzoek                     3.264704
politie                       3.256129
nl                            3.235322
artikel                       3.205896
sr                            3.088744
figuur                        3.038443
slachtoffersm                 3.012357
prostitutie       

In [ ]:
pickle.dump(vec_nl, open("../data/vec_nl.p", "wb"))
pickle.dump(vec_en, open("../data/vec_en.p", "wb"))